In [7]:
import numpy as np
import cv2
from scipy.fft import fft2, ifft2
from scipy.fftpack import dct, idct
import os
from skimage.msetrics import peak_signal_noise_ratio, structural_similarity



# Convert the image to gray scale (monochromatic) image using an in-built function


In [8]:
# Load the image
image = cv2.imread('input.jpg', cv2.IMREAD_COLOR)

# Convert to grayscale
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#gray_image2=cv2.resize(gray_image,(200,200))

cv2.imshow("gray", gray_image2)
cv2.imwrite('gray.jpg',gray_image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Division of image to blocks (sub images)


In [9]:
# Define block size (e.g., 8x8)
block_size =200
height, width = gray_image2.shape
blocks = []

for y in range(0, height, block_size):
    for x in range(0, width, block_size):
        block = gray_image2[y:y+block_size, x:x+block_size]
        blocks.append(block)


for i, block in enumerate(blocks):
    cv2.imwrite(f'block_{i}.jpg', block)

print(f'Saved {len(blocks)} blocks to image files.')

Saved 12 blocks to image files.


# Transform of blocks

In [10]:
transformed_blocks_dft = []
transformed_blocks_dct = []

for block in blocks:
    # Apply DFT
    dft_block = fft2(block)
    transformed_blocks_dft.append(dft_block)

    # Apply DCT
    dct_block = dct(dct(block, axis=0, norm='ortho'), axis=1, norm='ortho')
    transformed_blocks_dct.append(dct_block)


In [11]:
output_folder = r'C:\Users\win-10\Desktop\COV686\DFT'
for i, transformed_block in enumerate(transformed_blocks_dft):
    # Scale and shift the values of the transformed block for proper visualization
    transformed_block = np.abs(transformed_block)  # Take the absolute value
    transformed_block = np.log1p(transformed_block)  # Apply logarithm for better visualization
    transformed_block = cv2.normalize(transformed_block, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    file_path = os.path.join(output_folder, f'transformed_dft_block_{i}.jpg')
    cv2.imwrite(file_path, transformed_block)

print(f'Saved {len(transformed_blocks_dft)} transformed DFT blocks to {output_folder}.')

Saved 12 transformed DFT blocks to C:\Users\win-10\Desktop\COV686\DFT.


In [12]:
output_folder = r'C:\Users\win-10\Desktop\COV686\DCT'

for i, transformed_block in enumerate(transformed_blocks_dct):
    # Scale and shift the values of the transformed block for proper visualization
    transformed_block = np.abs(transformed_block)  # Take the absolute value
    transformed_block = np.log1p(transformed_block)  # Apply logarithm for better visualization
    transformed_block = cv2.normalize(transformed_block, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    file_path = os.path.join(output_folder, f'transformed_dct_block_{i}.jpg')
    cv2.imwrite(file_path, transformed_block)

print(f'Saved {len(transformed_blocks_dct)} transformed DCT blocks to {output_folder}.')

Saved 12 transformed DCT blocks to C:\Users\win-10\Desktop\COV686\DCT.


# Truncation or quantization (can try different strategies).

In [13]:
# Define a quantization step size
quantization_step = 10  # Adjust as needed

quantized_blocks_dft = []
quantized_blocks_dct = []

for dft_block, dct_block in zip(transformed_blocks_dft, transformed_blocks_dct):
    # Quantize DFT coefficients
    quantized_dft_block = np.round(dft_block / quantization_step) * quantization_step
    quantized_blocks_dft.append(quantized_dft_block)

    # Quantize DCT coefficients (can use a different step size)
    quantized_dct_block = np.round(dct_block / (quantization_step * 2)) * (quantization_step * 2)
    quantized_blocks_dct.append(quantized_dct_block)




# Inverse transform and reconstruction of image from the retained coefficents.

In [15]:
reconstructed_image_dft = np.zeros_like(gray_image2, dtype=np.float64)
reconstructed_image_dct = np.zeros_like(gray_image2, dtype=np.float64)

index = 0

for y in range(0, height, block_size):
    for x in range(0, width, block_size):
        # Inverse DFT
        inverse_dft_block = np.real(ifft2(quantized_blocks_dft[index]))
        reconstructed_image_dft[y:y+block_size, x:x+block_size] = inverse_dft_block

        # Inverse DCT
        inverse_dct_block = idct(idct(quantized_blocks_dct[index], axis=0, norm='ortho'), axis=1, norm='ortho')
        reconstructed_image_dct[y:y+block_size, x:x+block_size] = inverse_dct_block

        index += 1

# Convert the reconstructed images to uint8
reconstructed_image_dft = np.uint8(np.clip(reconstructed_image_dft, 0, 255))
reconstructed_image_dct = np.uint8(np.clip(reconstructed_image_dct, 0, 255))

# Display or save the reconstructed images
# cv2.imshow('Reconstructed Image (DFT)', reconstructed_image_dft)
# cv2.imshow('Reconstructed Image (DCT)', reconstructed_image_dct)
file_path1 = os.path.join(r'C:\Users\win-10\Desktop\COV686\DFT', f'Reconstructed Image (DFT).jpg')
cv2.imwrite(file_path1, reconstructed_image_dft)

file_path1 = os.path.join(r'C:\Users\win-10\Desktop\COV686\DCT', f'Reconstructed Image (DCT).jpg')
cv2.imwrite(file_path1, reconstructed_image_dct)


cv2.waitKey(0)
cv2.destroyAllWindows()


In [16]:
psnr_dft = peak_signal_noise_ratio(gray_image2, reconstructed_image_dft)
ssi_dft = structural_similarity(gray_image2, reconstructed_image_dft)
psnr_dct = peak_signal_noise_ratio(gray_image2, reconstructed_image_dct)
ssi_dct = structural_similarity(gray_image2, reconstructed_image_dct)

print(f'DFT: PSNR = {psnr_dft}, SSI = {ssi_dft}')
print(f'DCT: PSNR = {psnr_dct}, SSI = {ssi_dct}')

DFT: PSNR = 51.142931605940795, SSI = 0.9983892176601027
DCT: PSNR = 34.36183389230149, SSI = 0.9056244093070377
